## Read encoded track data and decode it
- Disc read data will invert on every read pulse (0->1, 1->0)
- This data represents how long the same bit value continued (bit interval, 000 -> 3, 11111 -> 5)
- The bit interval is encoded with `ord(' ')+bit_interval`. 
```sh
FDD output    :  111101110111110111011101
Captured data :  000011110000001111000011
This data     :     4   4     6   4   4 2
```

In [1]:
from floppy_shield import *

In [2]:
test = True

In [3]:
#track_data = 'FB30-0-10.txt'
#track_data = 'track2e-4m-a.txt'
#track_data = 'trk-00-0.txt'
#track_data = 'putty/t1.txt'
track_data = 'putty/cdos7-3.log'
track_data = 'putty/cdos7-77av.log'

bs = bitstream(track_data)

track = bs.disk['0-0']

if test:
    print(track)

[7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 12, 17, 12, 20, 23, 17, 16, 24, 23, 25, 23, 17, 16, 24, 24, 24, 23, 17, 16, 24, 24, 24, 23, 17, 16, 24, 24, 24, 23, 16, 17, 24, 24, 24, 23, 17, 16, 24, 24, 24, 23, 16, 17, 24, 24, 24, 23, 16, 17, 24, 24, 24, 23, 17, 15, 25, 24, 24, 23, 16, 16, 25, 24, 23, 24, 16, 16, 24, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 17, 16, 16, 15, 17, 15, 16, 16, 17, 15, 17, 15, 16, 16, 17, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 23, 32, 24, 32, 24, 16, 32, 24, 32, 24, 16, 32, 24, 32, 23, 17, 15, 17, 15, 17, 15, 17, 23, 17, 16, 16, 15, 17, 16, 16, 16, 16, 15, 17, 15, 17, 15, 17, 15, 17, 15, 17, 15, 16, 16, 25, 23, 16, 16, 17, 23, 32, 16, 32, 16, 16, 32, 32, 24, 16, 24, 24, 16, 16, 24, 24, 23, 17, 16, 24, 24, 24, 23, 

## Display histogram of bit interval
In case the reading clock is 4MHz, 1 represents 0.25us

In [4]:
def generate_histogram(data):
    histo = [ 0 ] * 256
    for l in data:
        if l>=len(histo):
            l = len(histo) - 1
        histo[l]+=1
    return histo

if False:
    histo = generate_histogram(track)
    for i,v in enumerate(histo):
        print(i, v)

%matplotlib inline
import matplotlib.pyplot as plt

if False:
    plt.figure(figsize=(8,4), dpi=200)
    plt.xlabel('count')
    plt.ylabel('frequency')
    #plt.xlim(0,80)
    plt.grid(True)
    plt.hist(track, bins=256, range=(0,100), histtype='stepfilled', orientation='vertical', log=False)
    plt.show()

    plt.figure(figsize=(8,4), dpi=200)
    plt.xlabel('count')
    plt.ylabel('frequency')
    #plt.xlim(0,80)
    plt.grid(True)
    plt.hist(track, bins=256, range=(0,100), histtype='stepfilled', orientation='vertical', log=True)
    plt.show()

In [5]:
# G(X) = 1+X~5+X~12+X~16
# 00*14*02*A1*A1 FE 01 01 03 01 DD EA

if test:
    # test
    crc = CCITT_CRC()

    # FE 01 01 03 01 DD EA
    crc.reset()
    data=[0xfe, 0x01, 0x01, 0x03, 0x01, 0xdd, 0xea]
    crc.data(data[:])
    print(format(crc.get(), '04X'))

    # FE 01 01 10 01 8B CA
    crc.reset()
    data=[0xfe, 0x01, 0x01, 0x10, 0x01, 0x8b, 0xca]
    crc.data(data[:])
    print(format(crc.get(), '04X'))

    # FB + FF*256 + FB E5 (FE)
    crc.reset()
    a  = [ 0xfb ]
    a += [ 0xff ] * 256
    a += [ 0xfb, 0xe5 ]
    crc.data(a)
    print(format(crc.get(), '04X'))

    crc.reset()
    crc.data([0xa1, 0xa1, 0xa1])
    print(format(crc.get(), '02X'))


0000
0000
0000
B88B


MFM spec
Clock bit is inserted when the previous and next data bit are both 0.
```
FF: ' D D D D D D D D'
00: '? C C C C C C C '
C2: ' D D  C C C  D  '
A1: ' D   D  C C C  D'
```
Missing clock
```
C2: ' D D  C * C  D  '
A1: ' D   D  C * C  D'
```

In [6]:
track = bs.disk['0-0']
id_buf = search_all_idam(track, high_gain=0.4, low_gain=0, log_level=1)

print(id_buf)

IDAM CRC - OK (00,00,01,05)
IDAM CRC - OK (00,ff,ff,ff)
IDAM CRC - OK (00,c6,f7,b6)
IDAM CRC - OK (00,00,02,04)
IDAM CRC - OK (00,00,03,05)
IDAM CRC - OK (00,00,04,05)
IDAM CRC - OK (00,d7,f5,a6)
IDAM CRC - OK (00,00,06,05)
IDAM CRC - OK (92,72,f7,e8)
IDAM CRC - OK (00,00,08,05)
IDAM CRC - OK (00,00,09,05)
IDAM CRC - OK (00,00,0a,05)
IDAM CRC - OK (00,00,0b,05)
IDAM CRC - OK (00,00,0c,05)
IDAM CRC - OK (00,00,0d,05)
IDAM CRC - OK (00,00,0e,05)
IDAM CRC - OK (00,00,0f,05)
IDAM CRC - OK (00,00,10,05)
[[0, 0, 1, 5, True], [0, 255, 255, 255, True], [0, 198, 247, 182, True], [0, 0, 2, 4, True], [0, 0, 3, 5, True], [0, 0, 4, 5, True], [0, 215, 245, 166, True], [0, 0, 6, 5, True], [146, 114, 247, 232, True], [0, 0, 8, 5, True], [0, 0, 9, 5, True], [0, 0, 10, 5, True], [0, 0, 11, 5, True], [0, 0, 12, 5, True], [0, 0, 13, 5, True], [0, 0, 14, 5, True], [0, 0, 15, 5, True], [0, 0, 16, 5, True]]


## Decode IBM format

In [7]:
if test:
    #track, mfm_buf, mc_buf, er = decodeFormat(interval_buf, high_gain=0.01, low_gain=0.01, verbose=True)
    trk, mfm_buf, mc_buf, sct_read, sct_err = decodeFormat(track, high_gain=0.35, low_gain=0.025, log_level=0)
    print('{} sectors found. {} errors'.format(sct_read, sct_err))
    for sec in trk:
        print(sec[0])
        dump_list_hex(sec[2])

2 sectors found. 1 errors
[0, 0, 1, 5]
[ 0x20, 0x46, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x19, 0x00, 0x18, 0x00, 0x0E, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x80, 0x4F, 0xC7, 0x52, 0x00, 0x00, 0x00, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x04, 0x00, 0x00, 0x8E, 0x00, 0x00, 0x00, 0x0A, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x86, 0xFD, 0x1F, 0x8B, 0x96, 0x0F, 0x10, 0xCE, 0xFC, 0x7F, 0xD6, 0x04, 0xC4, 0x02, 0x26, 0x0F, 0x4F, 0xFD, 0x03, 0x00, 0xC6, 0x41, 0xD7, 0x03, 0x1F, 0x8B, 0xBE, 0xFB, 0xFC, 0x6E, 0x84, 0x1A, 0x50, 0x10, 0xCE, 0x01, 0x00, 0x30, 0x8C, 0xD0, 0x33, 0x8D, 0x00, 0x8C, 0xEF, 0x02, 0x53, 0xC0, 0x08, 0xE7, 0x05, 0x86, 0x05, 0xAD, 0x9F, 0xFB, 0xFA, 0x24, 0x4A, 0x4A, 0x27, 0x04, 0x8D, 0x10, 0x24, 0xF3, 0xC6, 0x29, 0xF7, 0xFD, 0x03, 0x1F, 0x8B, 0x8E, 0x80, 0x00, 0x6E, 0x9F, 0xFB, 0xFE, 0x34, 0x20, 0x6A, 0x84, 0x6A, 0x84, 0x8D, 0x18,

In [9]:
# C2 C2 C2 FC : Index
# A1 A1 A1 FE : AM1 (ID)
# A1 A1 A1 FB : AM2 (DAM)
# A1 A1 A1 F8 : AM2 (DDAM)
# MB8876
# $F5 -> $A1
# $F6 -> $C2
# $F6 -> CRC

dumpMFM(mfm_buf, mc_buf)
#dump_list_hex(mfm_buf)

  FF  FF  39  39  39  39  39  39  39  39  39  39  38  00  00  00  00  00  00  00  00  00  00  00  02 *04 *02 *A1 *A1  FE  00  00
  01  05  BA  88  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  4E  00  00  00  00  00  00
  00  00  00  00  00  00  00 *14 *02 *A1 *A1  FB  20  46  00  00  00  00  00  00  00  00  00  00  00  19  00  18  00  0E  00  01
  00  00  00  00  80  4F  C7  52  00  00  00  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04  04
  04  04  04  04  04  04  00  00  8E  00  00  00  0A  00  00  00  00  00  00  00  86  FD  1F  8B  96  0F  10  CE  FC  7F  D6  04
  C4  02  26  0F  4F  FD  03  00  C6  41  D7  03  1F  8B  BE  FB  FC  6E  84  1A  50  10  CE  01  00  30  8C  D0  33  8D  00  8C
  EF  02  53  C0  08  E7  05  86  05  AD  9F  FB  FA  24  4A  4A  27  04  8D  10  24  F3  C6  29  F7  FD  03  1F  8B  8E  80  00
  6E  9F  FB  FE  34  20  6A  84  6A  84  8D  18  6C  84  6C  84  10  8E  40  00  31  3F  26  FC 

In [9]:
#disk_data = 'fb30-00-0.txt'
#disk_data = 'protect_killer.txt'
#disk_data = 'track10-4m-a.txt'
#disk_data = 'track2e-4m-a.txt'
#disk_data = 'track0-1m.txt'
#disk_data = 'fbasic30.txt'
#disk_data = 'fb30a.txt'
#disk_data = 'trk-01-0.txt'
#disk_data = 'putty/8-525.log'
#disk_data = 'fb30-0-10.txt'
#disk_data = 'putty/F-BASIC30-525.log'
#disk_data = 'putty/OS-9_L1-2_Programming_Tools_1.log'
#disk_data = 'putty/Modula-2_09_1.log'
disk_data = 'putty/Protect_Killer.log'

from mpl_toolkits.mplot3d import Axes3D

bs = bitstream()
bs.open(disk_data)

for track_id in bs.disk:
    interval = bs.disk[track_id]
    trk, mfm_buf, mc_buf, sct_read, sct_err = decodeFormat(interval, clk_spd=4e6, high_gain=0., low_gain=0., log_level=0)
            #track, mfm_buf, mc_buf, er = decodeFormat(interval_buf, clk_spd=4e6, high_gain=0.3, low_gain=0.025, verbose=False, ignore_CRC_error=False)
            #track, mfm_buf, mc_buf, er = decodeFormat(interval_buf, clk_spd=4e6, high_gain=0.001, low_gain=0.15, verbose=False, ignore_CRC_error=False)
    try:
        err = sct_err / (sct_read+sct_err)
    except DivisionByZero:
        err = 0
    print(track_id, sct_read, sct_err, err)


0-0 2 1 0.3333333333333333
1-1 0 1 1.0
2-0 0 0 0
3-1 0 0 0
4-0 0 1 1.0
5-1 0 0 0
6-0 16 0 0.0
7-1 16 0 0.0
8-0 16 0 0.0
9-1 16 0 0.0
10-0 16 0 0.0
11-1 16 0 0.0
12-0 16 0 0.0
13-1 16 0 0.0
14-0 16 0 0.0
15-1 16 0 0.0
16-0 16 0 0.0
17-1 16 0 0.0
18-0 16 0 0.0
19-1 16 0 0.0
20-0 16 0 0.0
21-1 16 0 0.0
22-0 16 0 0.0
23-1 16 0 0.0
24-0 16 0 0.0
25-1 16 0 0.0
26-0 16 0 0.0
27-1 16 0 0.0
28-0 16 0 0.0
29-1 16 0 0.0
30-0 16 0 0.0
31-1 16 0 0.0
32-0 16 0 0.0
33-1 16 0 0.0
34-0 16 0 0.0
35-1 16 0 0.0
36-0 16 0 0.0
37-1 16 0 0.0
38-0 16 0 0.0
39-1 16 0 0.0
40-0 16 0 0.0
41-1 16 0 0.0
42-0 16 0 0.0
43-1 16 0 0.0


KeyboardInterrupt: 

In [ ]:
for track in disk:
    for sect in track:
        print(sect[0])

In [ ]:
for track in disk:
    for sect in track:
        for c in sect[2]:
            if c>=ord(' ') and c<=ord('z'):
                print(chr(c), end='')

In [ ]:
dumpMFM(mfm_buf, mc_buf)

In [ ]:
a=[0xA1, 0xA1, 0xA1, 0xFB, 
   0xAB, 0x8D, 0x53, 0x8E, 0x71, 0xBA, 0x8D, 0x4E, 0x8D, 0x4F, 0xBD, 0xDB, 0x49, 0xBD, 0xD8, 0x07,
   0x25, 0xEA, 0x9F, 0xD9, 0xC6, 0x02, 0x6D, 0x01, 0x27, 0x04, 0x8D, 0x40, 0x25, 0xDE, 0x5A, 0x2B, 
   0xDB, 0xC1, 0x03, 0x22, 0xD7, 0xF7, 0x71, 0xFA, 0x8D, 0x2F, 0x8E, 0x71, 0xAB, 0x8D, 0x27, 0x8E,
   0x71, 0xC8, 0x8D, 0x22, 0x8D, 0x23, 0xBD, 0xDB, 0x49, 0xBD, 0xD8, 0x07, 0x25, 0xEA, 0x9F, 0xD9,
   0xC6, 0x01, 0x6D, 0x01, 0x27, 0x04, 0x8D, 0x14, 0x25, 0xDE, 0x5D, 0x2B, 0xDB, 0xC1, 0x0F, 0x22,
   0xD7, 0xF7, 0x71, 0xFB, 0x20, 0x2E, 0x7E, 0x74, 0xA6, 0x7E, 0xD9, 0x2F, 0x5F, 0x34, 0x06, 0x9D,
   0xD2, 0x26, 0x05, 0x00, 0x03, 0xC7, 0x08, 0x06, 0x10, 0x01, 0x03, 0xC6, 0x43, 0x21, 0xFE, 0x38,
   0x02, 0x02, 0x42, 0x23, 0x86, 0x3C, 0x30, 0x00, 0x02, 0x4C, 0x9F, 0xC2, 0x03, 0x42, 0x09, 0x01,
   0xF8, 0x0F, 0x1C, 0x1F, 0xC8, 0x48, 0x03, 0xE0, 0x00, 0xC0, 0x00, 0x10, 0xC0, 0x18, 0x82, 0x4F,
   0xE4, 0x10, 0x18, 0x00, 0x40, 0x00, 0xC0, 0x78, 0x82, 0x4F, 0xCE, 0x06, 0x10, 0xC0, 0x00, 0x10,
   0xC0, 0x00, 0x07, 0xC8, 0x82, 0x4F, 0xCE, 0x00, 0x18, 0x00, 0x08, 0x83, 0xC0, 0x08, 0x86, 0x10,
   0xC4, 0x82, 0x1F, 0xC0, 0x43, 0x1E, 0x00, 0x07, 0x04, 0x00, 0x10, 0xC0, 0x67, 0x13, 0xE1, 0x00,
   0x0E, 0x44, 0x7F, 0x9F, 0xC0, 0x1E, 0x11, 0x9C, 0x19, 0xC0, 0x1C, 0x8E, 0x20, 0x00, 0x1F, 0x30,
   0x4C, 0x20, 0x19, 0x00, 0x1F, 0x30, 0x00, 0x1F, 0x20, 0x0C, 0x20, 0x01, 0xC0, 0x1F, 0x21, 0xC0,
   0x1F, 0x27, 0x0C, 0x20, 0x92, 0x40, 0x1F, 0x27, 0x00, 0x1F, 0x30, 0x0C, 0x20, 0xC0, 0x00, 0x1F,
   0x27, 0xC0, 0x1F, 0x26, 0x0C, 0x20, 0x80, 0x80, 0x1F, 0x26, 0x00, 0x1F, 0x24, 0xCC, 0x20, 0x00,
   0xC0, 0x1F]
b = [ 0xA1, 0xA1, 0xA1, 0xFE, 0x00, 0x01, 0x01, 0x01, 0xCD, 0x3C ]
c = [                   0xFE, 0x00, 0x01, 0x01, 0x01, 0xCD, 0x3C ]
d = [ 0xA1, 0xA1, 0xA1, 0xFE, 0x00, 0x01, 0x01, 0x01, 0x00, 0x00 ]
e = [                   0xFE, 0x14, 0x00, 0x01, 0x01, 0x2B, 0x5A ]
crc = CCITT_CRC()
#crc.reset()
crc.reset()
crc.data(a)
print(format(crc.get(), '04X'))

In [ ]:
class d77_image:
    def __init__(self):
        pass
    
    def create_header(self, disk_name, wp_flag=0, disk_type=0):
        """
        Args:
          wp_flag = 0: no protect, 0x10: write protect
          disk_type = 0x00:2D, 0x10:2DD, 0x20:2HD
        """
        hdr = bytearray([0]*0x02b0)
        for p, c in enumerate(disk_name):
            hdr[p] = ord(c)
        hdr[0x1a] = wp_flag
        hdr[0x1b] = disk_type
        return hdr

    def create_sector(self, sect_data, c, h, r, n, num_sec, density=0, am=0, status=0):
        """
        Args:
          sect_data (list): sector data
          c, h, r, n: C/H/R/N (sector ID)
          num_sec: number sectors in this track
          density: 0x00: double, 0x40: single
          am: Address mark (0x00: Data Mark, 0x10: DDM)
          status: Disk BIOS status (0==no error)
        """
        sect_size = len(sect_data)
        sect = bytearray([c, h, r, n, num_sec % 0x100, num_sec // 0x100, density, am, status, 0, 0, 0, 0, 0, sect_size % 0x100, sect_size // 0x100 ])
        sect += bytearray(sect_data)
        return sect

    def set_dword(self, barray, pos, data):
        for i in range(4):
            barray[pos+i] = data & 0xff
            data >>= 8

    def set_word(self, barray, pos, data):
        for i in range(2):
            barray[pos+i] = data & 0xff
            data >>= 8

    def set_track_table(self, hdr, track_num, data):
        pos = 0x20 + track_num*4
        self.set_dword(hdr, pos, data)

    def generate(self, disk_data, disk_name = 'DISK'):
        img = self.create_header(disk_name)
        for track in disk:
            num_sects = len(track)
            if num_sects==0:
                continue
            track_num = track[0][0][0]*2 + track[0][0][1]  # C*2 + H
            self.set_track_table(img, track_num, len(img))
            for sect in track:
                idam = sect[0]
                sect_data = self.create_sector(sect[2], idam[0], idam[1], idam[2], idam[3], num_sects, density=0, am=0 if sect[3] else 0x10, status=0 if sect[1] else 0x80)
                img += bytearray(sect_data)
        # set total disk image size
        self.set_dword(img, 0x001c, len(img))
        return img

d77 = d77_image()
img = d77.generate(disk)
with open('disk_img.d77', 'wb') as f:
    f.write(img)
